# CVC Data Summaries (with simple method hydrology)

## Setup the basic working environment

In [ ]:
%matplotlib inline

import os
import sys
import datetime
import warnings

import numpy as np
import matplotlib.pyplot as plt
import pandas
import seaborn
seaborn.set(style='ticks', context='paper')

import wqio
from wqio import utils
import pybmpdb
import pynsqd

import pycvc

min_precip = 1.9999
big_storm_date = datetime.date(2013, 7, 8)

pybmpdb.setMPLStyle()
seaborn.set(style='ticks', rc={'text.usetex': False}, palette='deep')

POCs = [p['cvcname'] for p in filter(lambda p: p['include'], pycvc.info.POC_dicts)]

if wqio.testing.checkdep_tex() is None:
    tex_msg = ("LaTeX not found on system path. You will "
               "not be able to compile ISRs to PDF files")
    warnings.warn(tex_msg, UserWarning)
    
warning_filter = "ignore" 
warnings.simplefilter(warning_filter)

## Load tidy data
Data using the Simple Method hydrology is suffixed with `_simple`.
You could also use the SWMM Model hydrology with the `_SWMM` files.

In [ ]:
# simple method file
tidy_file = "output/tidy/hydro_simple.csv"


# # SWMM Files
# tidy_file = "output/tidy/hydro_swmm.csv"

hydro = pandas.read_csv(tidy_file, parse_dates=['start_date', 'end_date'])

## High-level summaries

### Hydrologic info and stats
Does not include the July 8, 2013 storm event.

For LV-1 and LV-2, event durations are winsorized to replace outliers beyond the 97.5 percentile.

For more information, see:

  1. [scipy.stats.mstats.winsorize](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.winsorize.html)
  2. [wqio.utils.winsorize_dataframe](https://github.com/Geosyntec/wqio/blob/6a056bee34f9c3ed8e300c3d6db1130a6f9ce980/wqio/utils/misc.py#L1548)

In [ ]:
def winsorize_duration(g):    
    winsor_limits = {
        'ED-1': (0.0, 0.0),
        'LV-1': (0.2, 0.1),
        'LV-2': (0.2, 0.3),
        'LV-4': (0.0, 0.0),
    }
    return wqio.utils.winsorize_dataframe(g, duration_hours=winsor_limits[g.name])

with pandas.ExcelWriter("output/xlsx/CVCHydro_StormInfo_Simple.xlsx") as xl_storminfo:
    
    hydro.to_excel(xl_storminfo, sheet_name='Storm Info', index=False)
    for timegroup in [None, 'year', 'season', 'grouped_season']:
        stat_options = {
            'minprecip': min_precip,
            'groupby_col': timegroup,
        }

        (
            hydro.groupby('site')
                 .apply(winsorize_duration)
                 .pipe(pycvc.summary.remove_load_data_from_storms, [big_storm_date], 'start_date')
                 .pipe(pycvc.summary.storm_stats, **stat_options)
                 .to_excel(xl_storminfo, sheet_name='Storm Stats - {}'.format(timegroup), index=False)
                 
        )

### Hydrologic Pairplots 
Expected failures due to lack of data:
  1. LV-2, outflow
  1. LV-4, grouped_season

In [ ]:
for site in ['ED-1', 'LV-2', 'LV-4']:
    for by in ['year', 'outflow', 'season', 'grouped_season']:
        try:               
            pycvc.viz.hydro_pairplot(hydro, site, by=by)
        except:              
            print('failed on {}, {}'.format(site, by))

### Hydrologic joint distribution plots

In [ ]:
sites = [
    {'name': 'ED-1', 'color': seaborn.color_palette()[0]},
    {'name': 'LV-1', 'color': seaborn.color_palette()[1]},
    {'name': 'LV-2', 'color': seaborn.color_palette()[4]},
    {'name': 'LV-4', 'color': seaborn.color_palette()[5]},
]
for site in sites:   
    pycvc.viz.hydro_jointplot(
        hydro=hydro, site=site['name'],
        xcol='total_precip_depth', 
        ycol='outflow_mm', 
        conditions="outflow_mm > 0", 
        one2one=True,
        color=site['color'],
    )

    pycvc.viz.hydro_jointplot(
        hydro=hydro, site=site['name'],
        xcol='antecedent_days', 
        ycol='outflow_mm', 
        conditions="outflow_mm > 0", 
        one2one=False,
        color=site['color'],
    )

    pycvc.viz.hydro_jointplot(
        hydro=hydro, site=site['name'],
        xcol='total_precip_depth', 
        ycol='antecedent_days', 
        conditions="outflow_mm == 0", 
        one2one=False,
        color=site['color'],
    )
    
    pycvc.viz.hydro_jointplot(
        hydro=hydro, site=site['name'],
        xcol='peak_precip_intensity', 
        ycol='peak_outflow', 
        conditions=None, 
        one2one=False,
        color=site['color'],
    )
    
    plt.close('all')